## Projet PACTE 4A-IIIA: Data Science Formula 1 World  Championship
### Sujet: L'impact de la position de depart sur le resultat d'une course.

Dans cette étude, nous nous intéressons à l’impact de la position de départ sur le résultat d’une course de Formule 1, en adoptant une approche basée sur l’analyse descriptive des données. L’objectif est de comprendre comment la position de départ influence les performances des pilotes, tout en explorant des facteurs connexes tels que les classements cumulés et les performances individuelles.

Pour cette analyse, nous utilisons plusieurs fichiers issus de la base de données Formula 1, chacun apportant des informations complémentaires :  

- **`lap_times.csv`** : Contient les temps au tour pour chaque pilote, permettant d’examiner la régularité et la performance au cours de la course.  
- **`qualifying.csv`** : Fournit les résultats des qualifications, y compris les temps enregistrés dans chaque session, et aide à relier les positions de départ aux performances en course.  
- **`races.csv`** : Offre des informations contextuelles sur chaque Grand Prix, telles que l’année, le lieu, et le circuit.  
- **`results.csv`** : Détaille les résultats spécifiques de chaque course, comme la position finale, les points marqués et les incidents de course.  
- **`status.csv`** : Indique les raisons de l’abandon ou de l’échec pour certains pilotes, utile pour analyser les événements qui perturbent les performances.  
- **`driver_standings.csv`** : Résume le classement global des pilotes après chaque course, incluant les points accumulés et les victoires totales. Ce fichier nous permet d’intégrer une perspective cumulative pour évaluer l’impact des résultats individuels sur le championnat.  

Les variables pertinentes que nous analyserons incluent :  
1. **`grid`** (fichier `results.csv`) : Position de départ sur la grille, essentielle pour comprendre son influence sur les performances en course.  
2. **`positionOrder`** (fichier `results.csv`) : Position finale à l’arrivée, pour évaluer les progrès ou les déclins des pilotes pendant la course.  
3. **`points`** (fichier `driver_standings.csv`) : Points cumulés par pilote, pour relier les performances individuelles à l’évolution globale du championnat.  
4. **`milliseconds`** (fichier `lap_times.csv`) : Temps au tour en millisecondes, pour analyser la constance des pilotes et détecter les variations de performance.  
5. **`q3`** (fichier `qualifying.csv`) : Temps réalisé lors de la troisième session de qualification, clé pour déterminer la position de départ.  
6. **`status`** (fichier `status.csv`) : Raison de l’abandon ou du résultat, pour comprendre les incidents et leur impact sur les classements.  

En analysant ces variables, nous espérons identifier des tendances et corrélations qui éclairent le rôle des qualifications, des performances en course, et des incidents dans les résultats finaux.  

In [2]:
import pandas as pd

#### 1. Chargement des données

In [54]:
lap_times = pd.read_csv("./dataset/lap_times.csv")
qualifying = pd.read_csv("./dataset/qualifying.csv") 
races = pd.read_csv("./dataset/races.csv")
results = pd.read_csv("./dataset/results.csv")
status = pd.read_csv("./dataset/status.csv")
driver_standings = pd.read_csv("./dataset/driver_standings.csv")

##### Verification de chargement

In [55]:
# Vérification des datasets importés
datasets = {
    "lap_times": lap_times,
    "qualifying": qualifying,
    "races": races,
    "results": results,
    "status": status,
    "driver_standings": driver_standings
}

for name, df in datasets.items():
    print(f"{name}: {df.shape[0]} lignes, {df.shape[1]} colonnes")

lap_times: 575029 lignes, 6 colonnes
qualifying: 10254 lignes, 9 colonnes
races: 1125 lignes, 18 colonnes
results: 26519 lignes, 18 colonnes
status: 139 lignes, 2 colonnes
driver_standings: 34595 lignes, 7 colonnes


#### 2. Préparation des données

#### 2.1. Exploration initiale des données
Cette étape permet d’avoir une vue d’ensemble des datasets (colonnes, types de données, valeurs manquantes).

In [56]:
lap_times.head()

,raceId,driverId,lap,position,time,milliseconds
0,841,20,1,1,1:38.109,98109
1,841,20,2,1,1:33.006,93006
2,841,20,3,1,1:32.713,92713
3,841,20,4,1,1:32.803,92803
4,841,20,5,1,1:32.342,92342


In [57]:
qualifying.head(15)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
5,6,18,15,7,11,6,1:26.427,1:26.101,1:28.527
6,7,18,3,3,7,7,1:26.295,1:26.059,1:28.687
7,8,18,14,9,9,8,1:26.381,1:26.063,1:29.041
8,9,18,10,7,12,9,1:26.919,1:26.164,1:29.593
9,10,18,20,5,15,10,1:26.702,1:25.842,\N


Convertion des variables temporelles q1, q2, q3 en secondes

In [58]:
def convert_time_to_seconds(time_str):
    """
    Convertit une chaîne de temps 'mm:ss.SSS' en secondes.
    Ignore les valeurs manquantes.
    """
    if not str(time_str) or str(time_str) == "\\N":  # Vérifie si la valeur est manquante
        return None  # Retourne vide si la valeur est manquante
    try:
        # Divise la chaîne en minutes et secondes
        minutes, seconds = str(time_str).split(':')
        # Convertit la partie minutes en entier et la partie secondes en flottant
        total_seconds = int(minutes) * 60 + float(seconds)
        return total_seconds
    except ValueError:
        # Gestion des erreurs au cas où la chaîne est mal formatée
        return None
    
# Conversion des colonnes de temps
time_columns = ['q1', 'q2', 'q3']
for col in time_columns:
    qualifying[col + "new"] = [convert_time_to_seconds(t) for t in qualifying[col]]
    

In [59]:
qualifying.head(20)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,q1new,q2new,q3new
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,86.572,85.187,86.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,86.103,85.315,86.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,85.664,85.452,87.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,85.994,85.691,87.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,85.960,85.518,87.236
5,6,18,15,7,11,6,1:26.427,1:26.101,1:28.527,86.427,86.101,88.527
6,7,18,3,3,7,7,1:26.295,1:26.059,1:28.687,86.295,86.059,88.687
7,8,18,14,9,9,8,1:26.381,1:26.063,1:29.041,86.381,86.063,89.041
8,9,18,10,7,12,9,1:26.919,1:26.164,1:29.593,86.919,86.164,89.593
9,10,18,20,5,15,10,1:26.702,1:25.842,\N,86.702,85.842,NaN


In [60]:
races.head()

,raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N


In [61]:
results.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
0,1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
1,2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
2,3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
3,4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
4,5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1


In [62]:
status.head()

,statusId,status
0,1,Finished
1,2,Disqualified
2,3,Accident
3,4,Collision
4,5,Engine


In [63]:
driver_standings.head()

,driverStandingsId,raceId,driverId,points,position,positionText,wins
0,1,18,1,10.0,1,1,1
1,2,18,2,8.0,2,2,0
2,3,18,3,6.0,3,3,0
3,4,18,4,5.0,4,4,0
4,5,18,5,4.0,5,5,0


In [64]:
# Afficher les premières lignes et résumés des datasets
for name, df in datasets.items():
    print(f"Dataset: {name}")
    print(df.info())


Dataset: lap_times
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 575029 entries, 0 to 575028
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   raceId        575029 non-null  int64 
 1   driverId      575029 non-null  int64 
 2   lap           575029 non-null  int64 
 3   position      575029 non-null  int64 
 4   time          575029 non-null  object
 5   milliseconds  575029 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 26.3+ MB
None
Dataset: qualifying
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10254 entries, 0 to 10253
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   qualifyId      10254 non-null  int64  
 1   raceId         10254 non-null  int64  
 2   driverId       10254 non-null  int64  
 3   constructorId  10254 non-null  int64  
 4   number         10254 non-null  int64  
 5   position       10254 non-null

##### 2.2. Traitement des valeurs manquantes
Les valeurs manquantes peuvent fausser les résultats. On identifie ces valeurs et choisit une stratégie : suppression, imputation, ou autre.

In [71]:
# Vérification des valeurs manquantes
for name, df in datasets.items():
    print(f"{name} - Valeurs manquantes:\n{df.isnull().sum()}\n")

lap_times - Valeurs manquantes:
raceId          0
driverId        0
lap             0
position        0
time            0
milliseconds    0
dtype: int64

qualifying - Valeurs manquantes:
qualifyId         0
raceId            0
driverId          0
constructorId     0
number            0
position          0
q1                0
q2               13
q3               27
q1new             0
q2new             0
q3new             0
dtype: int64

races - Valeurs manquantes:
raceId         0
year           0
round          0
circuitId      0
name           0
date           0
time           0
url            0
fp1_date       0
fp1_time       0
fp2_date       0
fp2_time       0
fp3_date       0
fp3_time       0
quali_date     0
quali_time     0
sprint_date    0
sprint_time    0
dtype: int64

results - Valeurs manquantes:
resultId           0
raceId             0
driverId           0
constructorId      0
number             0
grid               0
position           0
positionText       0
positionOrder

##### 2.3. Correction des valeurs maquantes

+ qualifying.csv

In [67]:
qualifying['q1new'] = qualifying['q1new'].fillna(qualifying['q1new'].mean())
qualifying['q2new'] = qualifying['q2new'].fillna(qualifying['q2new'].mean())
qualifying['q3new'] = qualifying['q3new'].fillna(qualifying['q3new'].mean())

In [68]:
qualifying.head(20)

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3,q1new,q2new,q3new
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714,86.572,85.187000,86.714000
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869,86.103,85.315000,86.869000
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079,85.664,85.452000,87.079000
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178,85.994,85.691000,87.178000
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236,85.960,85.518000,87.236000
5,6,18,15,7,11,6,1:26.427,1:26.101,1:28.527,86.427,86.101000,88.527000
6,7,18,3,3,7,7,1:26.295,1:26.059,1:28.687,86.295,86.059000,88.687000
7,8,18,14,9,9,8,1:26.381,1:26.063,1:29.041,86.381,86.063000,89.041000
8,9,18,10,7,12,9,1:26.919,1:26.164,1:29.593,86.919,86.164000,89.593000
9,10,18,20,5,15,10,1:26.702,1:25.842,\N,86.702,85.842000,87.415273


#### 3. Analyse descriptive des données

##### 3.1. Statistiques descriptives
Fournir des statistiques (moyenne, médiane, écart-type, etc.) pour comprendre la distribution des variables.

+ **`q3`** (fichier `qualifying.csv`) : Temps réalisé lors de la troisième session de qualification, clé pour déterminer la position de départ.  
On va utiliser la nouvelle variable `q3new` à cet effet

In [ ]:
qualifying['q3new']